In [ ]:
!pip install sacrebleu
!pip install datasets
!pip install tqdm
!pip install numpy
!pip install huggingface_hub
!pip install faster-whisper -q
!pip install ctranslate2==4.6.0 -q

ERROR: Could not find a version that satisfies the requirement ctranslate2==4.6.0 (from versions: 2.17.0, 2.18.0, 2.19.0, 2.19.1, 2.20.0, 2.21.0, 2.21.1, 2.22.0, 2.23.0, 2.24.0, 3.0.0, 3.0.1, 3.0.2, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.5.1, 3.6.0, 3.7.0, 3.8.0, 3.9.0, 3.9.1, 3.10.0, 3.10.1, 3.10.2, 3.10.3, 3.11.0, 3.12.0, 3.13.0, 3.14.0, 3.15.0, 3.15.1, 3.16.0, 3.16.1, 3.17.0, 3.17.1, 3.18.0, 3.19.0, 3.20.0, 3.21.0, 3.22.0, 3.23.0, 3.24.0, 4.0.0, 4.1.0, 4.2.1, 4.3.1, 4.4.0, 4.5.0)
ERROR: No matching distribution found for ctranslate2==4.6.0


In [ ]:
!ct2-transformers-converter --model cobrayyxx/whisper-small-indo-eng --output_dir cobrayyxx/whisper-small-indo-eng-ct2 --copy_files tokenizer.json preprocessor_config.json --quantization float16

2024-12-16 15:42:50.051897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 15:42:50.071229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 15:42:50.077189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 15:42:51.219984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
vocab.json: 100% 836k/836k [00:00<00:00, 13.2MB/s]
tokenizer.json: 100% 3.93M/3.93M [00:00<00:00, 14.8MB/s]
preprocessor_config.json: 100% 339/339 [00:00<00:00, 2.66MB/s]


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from faster_whisper import WhisperModel
from sacrebleu.metrics import BLEU, CHRF, TER
from datasets import load_dataset, Dataset, Audio, DatasetDict
from tqdm import tqdm
import numpy as np


In [ ]:
fleurs_dataset = load_dataset("cobrayyxx/FLEURS_INDO-ENG_Speech_Translation",trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def predict(audio_array):
    model_name = "small"  # pretrained model - try "tiny", "base", "small", or "medium"
    model = WhisperModel(model_name, device="cuda", compute_type="float16")

    segments, info = model.transcribe(audio_array,
                                      beam_size=5,
                                      language="en",
                                      vad_filter=True
                                      )
    return segments, info

def metric_calculation(dataset):
    val_data = fleurs_dataset["validation"]
    bleu = BLEU()
    chrf = CHRF()
    lst_pred = []
    lst_gold = []
    for data in tqdm(val_data):
        gold_standard = data["text_en"]
        gold_standard = gold_standard.lower().strip()
        audio_array = data["audio"]["array"]
        # Ensure it's 1D
        audio_array = np.ravel(audio_array)

        # Convert to float32 if necessary
        audio_array = audio_array.astype(np.float32)
        pred_segments, pred_info = predict(audio_array)
        prediction_text = " ".join(segment.text for segment in pred_segments).lower().strip()
        lst_pred.append(prediction_text)
        lst_gold.append([gold_standard])
    bleu_score = bleu.corpus_score(lst_pred, lst_gold).score
    chrf_score = chrf.corpus_score(lst_pred, lst_gold).score

    return bleu_score, chrf_score

In [ ]:
final_bleu_score, final_chrf_score   = metric_calculation(fleurs_dataset)

100%|██████████| 713/713 [12:06<00:00,  1.02s/it]


In [ ]:
final_bleu_score, final_chrf_score

(33.03164318013809, 52.71401535707202)

In [ ]:
def predict(audio_array):
    model_name = "cobrayyxx/whisper-small-indo-eng-ct2"  # pretrained model - try "tiny", "base", "small", or "medium"
    model = WhisperModel(model_name, device="cuda", compute_type="float16")

    segments, info = model.transcribe(audio_array,
                                      beam_size=5,
                                      language="en",
                                      vad_filter=True
                                      )
    return segments, info

def metric_calculation(dataset):
    val_data = fleurs_dataset["validation"]
    bleu = BLEU()
    chrf = CHRF()
    lst_pred = []
    lst_gold = []
    for data in tqdm(val_data):
        gold_standard = data["text_en"]
        gold_standard = gold_standard.lower().strip()
        audio_array = data["audio"]["array"]
        # Ensure it's 1D
        audio_array = np.ravel(audio_array)

        # Convert to float32 if necessary
        audio_array = audio_array.astype(np.float32)
        pred_segments, pred_info = predict(audio_array)
        prediction_text = " ".join(segment.text for segment in pred_segments).lower().strip()
        lst_pred.append(prediction_text)
        lst_gold.append([gold_standard])
    bleu_score = bleu.corpus_score(lst_pred, lst_gold).score
    chrf_score = chrf.corpus_score(lst_pred, lst_gold).score

    return bleu_score, chrf_score

In [ ]:
pretrain_bleu_score, pretrain_chrf_score   = metric_calculation(fleurs_dataset)

100%|██████████| 713/713 [11:29<00:00,  1.03it/s]


In [ ]:
pretrain_bleu_score, pretrain_chrf_score

(34.82352832757854, 61.4468135930913)

In [ ]:
!sudo rm -rf /root/.cache/huggingface/hub/
!sudo mkdir /root/.cache/huggingface/

mkdir: cannot create directory ‘/root/.cache/huggingface/’: File exists


In [ ]:
!sudo rm -rf  /content/whisper-small-idn-ct

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("sanchit-gandhi/whisper-small-hi")
processor = WhisperProcessor.from_pretrained("sanchit-gandhi/whisper-small-hi")

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]